<a href="https://colab.research.google.com/github/a8nguyen/4845_projects/blob/main/nlp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

import os
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU
import matplotlib.pyplot as plt
import seaborn as sns

from keras.utils import to_categorical
from numpy import argmax


import re
import os
import sys
import numpy as np
import time
import math
from datetime import datetime

### **Loading Dataset**

In [ ]:
def load_data(filePath):
  data_cwe = open(filePath, 'r').read().lower()
  return data_cwe

### **Defining Preprocessing Class**

In [ ]:
class Language_Translation_Model:
  def __init__(self, lang="") -> None:
    self.lang = lang
    self.all_Unique_Characters = None
    self.length_all_Unique_Characters = None
    self.length_max_sentence = None
    #self.sentenceArray_to_OHE = None
    self.char_to_num = None
    self.num_to_char = None
    # self.ASCII_Chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
  
  def sentence_to_array(self, sentence):
    sentenceArray = [self.char_to_num[char] for char in sentence]
    #padding = [0 for _ in range(self.length_all_Unique_Characters-len(sentenceArray))]
    #sentenceArray = np.array(sentenceArray+padding)
    return sentenceArray
  
  def sentenceArray_to_OHE(self, sentenceArray):
    sentenceArray_OHE = to_categorical(sentenceArray)
    (req_rows, req_cols) = (self.length_max_sentence, self.length_all_Unique_Characters)
    (actual_rows, actual_cols) = sentenceArray_OHE.shape
    sentenceArray_OHE = np.concatenate((sentenceArray_OHE, np.zeros((req_rows-actual_rows, actual_cols), dtype='float32')), axis=0)
    sentenceArray_OHE = np.concatenate((sentenceArray_OHE, np.zeros((req_rows, req_cols-actual_cols), dtype='float32')), axis=1)
    return sentenceArray_OHE

  def OHE_to_sentenceArray(self, sentence_array_OHE):
    sentenceArray = []
    for OHE in sentence_array_OHE:
      if sum(OHE)!=0:
        sentenceArray.append(argmax(OHE))
    return sentenceArray
  
  def preprocess_data(self, text):
    data = pd.DataFrame()
    # Cleaning --------------------------
    text = text.replace("\n", " ") 
    for char in [' .', ' ,', ' ;', ' ?', ' !', '( ', ' )']:
      text = text.replace(char, char.strip())
    text = text.replace("<s>", "")
    
    # =========================== CHARACTERS ===========================
    # Unique Characters -----------------
    self.all_Unique_Characters = list(set(text.replace("</s>", "")))
    self.all_Unique_Characters.sort()
    
    # Max Length of Unique Characters ---
    self.length_all_Unique_Characters = len(self.all_Unique_Characters)
    
    # Preparing Character to Number and vice-versa -----
    self.char_to_num = {char:num for num, char in enumerate(self.all_Unique_Characters)}
    self.num_to_char = {val:key for key, val in self.char_to_num.items()}

    # =========================== SENTENCES ===========================
    # Sentences -------------------------
    sentences = text.split("</s>")[:-1]
    sentences = [sentence.strip() for sentence in sentences]
    
    # Max Length of Sentences -----------
    self.length_max_sentence = max([len(sentence) for sentence in sentences])
    
    # Preparing OHE Array Variable ------
    #self.sentenceArray_to_OHE = np.zeros((self.length_max_sentence, self.length_all_Unique_Characters), dtype='float32')
    
    # =========================== DATAFRAME ===========================
    # Storing sentences in dataframe ----
    data['sentence'] = sentences
    # Converting Sentence to Array Numbers ----
    data['sentence_array'] = data['sentence'].apply(lambda x: self.sentence_to_array(x))
    # Converting Array Numbers to One-Hot-Encoding (OHE) ----
    data['sentence_array_OHE'] = data['sentence_array'].apply(lambda x: self.sentenceArray_to_OHE(x))

    return data

### **Process Input and Target datasets**

In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/train-source.txt"
train_input_data = load_data(filePath)

In [ ]:
Input_LTM = Language_Translation_Model()
Input_data = Input_LTM.preprocess_data(train_input_data[0:10000])
Input_data.head()

In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/train-target.txt"
train_target_data = load_data(filePath)

In [ ]:
Output_LTM = Language_Translation_Model()
Output_data = Output_LTM.preprocess_data(train_target_data[0:10000])
Output_data.head()

,sentence,sentence_array,sentence_array_OHE
0,"cinnte go leor, thiocfadh dóibh bás a fháil ar...","[15, 21, 25, 25, 30, 17, 0, 19, 26, 0, 23, 17,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,bhí sé follasach go raibh an poll sin ag foscl...,"[14, 20, 35, 0, 29, 34, 0, 18, 26, 23, 23, 13,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,d'fhéadfadh siad bás a fháil ar a bhruach agus...,"[16, 2, 18, 20, 34, 13, 16, 18, 13, 16, 20, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,thiocfadh dóibh fosta lámh a chur ina mbás féi...,"[30, 20, 21, 26, 15, 18, 13, 16, 20, 0, 16, 36...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,ina dhiaidh sin bhí rud éigin dochreidte agus ...,"[21, 25, 13, 0, 16, 20, 21, 13, 21, 16, 20, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
encoder_input_data = np.array([record for record in Input_data['sentence_array_OHE']])
decoder_input_data = np.array([np.concatenate((np.zeros((record.shape[0], 1), dtype='float32'), record[:, 1:]), axis=1) for record in Output_data['sentence_array_OHE']])
decoder_target_data = np.array([record for record in Output_data['sentence_array_OHE']])

### **Model Preparation & Training**

In [ ]:
from datetime import datetime

In [ ]:
class Model_LSTM:
  def __init__(self, Input_LTM, Output_LTM, num_samples=10000, latent_dim=256, batch_size = 128, epochs=1):
    self.batch_size = batch_size
    self.epochs = epochs
    self.latent_dim = latent_dim
    self.num_samples = num_samples
    self.Input_LTM = Input_LTM
    self.Output_LTM = Output_LTM

    encoder_inputs = Input(shape=(None, self.Input_LTM.length_all_Unique_Characters))
    encoder = LSTM(latent_dim, return_state=True) # latent_dim = 256
    encoder_outputs, state_h, state_c = encoder(encoder_inputs) # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None, Output_LTM.length_all_Unique_Characters))

    # We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(Output_LTM.length_all_Unique_Characters, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Defining Model ------------------
    self.model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Compiling Model ------------------
    self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    # For Prediction ---------------------
    self.encoder_model = Model(encoder_inputs, encoder_states)

    decoder_states_inputs = [Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    self.decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
  def train(self, encoder_input_data, decoder_input_data, decoder_target_data):
    startTime = datetime.now()

    self.model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data,
          batch_size=self.batch_size,
          epochs=self.epochs,
          validation_split=0.2)
    endTime = datetime.now()
    print("Training is Completed Successfully in time : ", endTime-startTime)
    
  def predict(self, input_seq):
    # Encode the input as state vectors.
    states_value = self.encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, self.Output_LTM.length_all_Unique_Characters)) # Output_LTM.length_all_Unique_Characters -> Number of output characters
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = self.Output_LTM.num_to_char[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > self.Output_LTM.length_max_sentence):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, self.Output_LTM.length_all_Unique_Characters))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
ModelLSTM = Model_LSTM(Input_LTM, Output_LTM)

In [ ]:
ModelLSTM.train(encoder_input_data, decoder_input_data, decoder_target_data)

1/1 [==============================] - 6s 6s/step - loss: 1.2373 - accuracy: 0.3386 - val_loss: 1.2449 - val_accuracy: 0.7151
Training is Completed Successfully in time :  0:00:07.825980


In [ ]:
input_sentence = Input_data.loc[0, 'sentence']
print(input_sentence)

output_sentence = Output_data.loc[0, 'sentence']
print(output_sentence)

cinnte go leór, thiocfadh dóbhtha bás a fhagháil ar imeall an phuill udaí.
cinnte go leor, thiocfadh dóibh bás a fháil ar imeall an phoill úd.


In [ ]:
input_record = np.array([Input_data.loc[0, 'sentence_array_OHE']])
print(input_record.shape)

(1, 234, 39)


In [ ]:
decoded_sentence = ModelLSTM.predict(input_record)

In [ ]:
print(decoded_sentence)

In [ ]:
output_record = Output_data.loc[0, 'sentence']
print(output_record)

cinnte go leor, thiocfadh dóibh bás a fháil ar imeall an phoill úd.


In [ ]:
print(len(decoded_sentence), len(output_record))

237 67


### **Evaluation**

**Loading Input_Test data**

In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/train-source.txt"
train_input_data = load_data(filePath)

test_Input_LTM = Language_Translation_Model()
Input_data = test_Input_LTM.preprocess_data(train_input_data[0:10000])
Input_data.head()